<div align="Center">

# Orthogonalization and Least Squares Methods
## Matrix Computations (AS1209)
#### Utkarsh Tailor (2022BTech106)
#### Swastik Kulshreshtha (2022BTech105)
#### Saurabh Saini (2022BTech093)
#### Rajat Paliwal (2022BTech081)
#### Rahul Yadav (2022BTech079)
### Institute of Engineering and Technology, JK Lakshmipat University

</div>
<hr>

##### Importing Libraries

In [ ]:
from typing import Callable

import numpy as np
from scipy.io import mmread
import plotly.graph_objects as go
from numpyarray_to_latex.jupyter import to_jup as prettyPrint

##### Utilities

In [ ]:
def getQR(
    mat: np.ndarray,
    method: Callable[[np.ndarray], tuple[np.ndarray, np.ndarray]],
    printPretty: bool = True
) -> tuple[np.ndarray, np.ndarray]:
    q, r = method(mat)
    if not printPretty:
        return q, r

    if max(q.shape) < 6:
        prettyPrint(q, prefix='Q=')
        prettyPrint(r, prefix='R=')
        prettyPrint(q @ r, prefix='QR=')
    else:
        print('Q=')
        print(q)
        print('R=')
        print(r)
        print('QR=')
        print(q @ r)
    return q, r

In [ ]:
COLOURS = ('#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf')

def showMatrixAsVectors(
    mat: np.ndarray,
    fig: go.Figure = None,
    names: tuple[str] = None,
    colours: tuple[str] = COLOURS
) -> None:
    showFig = False
    if fig is None:
        fig = go.Figure()
        showFig = True

    rows, cols = mat.shape
    if rows == 2:
        arrows = []
        for idx in range(cols):
            fig.add_scatter(
                x=(0, mat[0][idx]),
                y=(0, mat[1][idx]),
                mode='lines',
                name=f'Vector {idx+1}' if names is None else names[idx],
                line=dict(color=colours[idx])
            )
            arrows.append(dict(
                x=mat[0][idx], y=mat[1][idx],
                xref='x', yref='y',
                axref='x', ayref='y',
                ax=0, ay=0,
                text='',
                showarrow=True,
                arrowhead=3,
                arrowwidth=1.5,
                arrowcolor=colours[idx]
            ))
        fig.update_layout(annotations=arrows)
    elif rows == 3:
        for idx in range(cols):
            fig.add_scatter3d(
                x=(0, mat[0][idx]),
                y=(0, mat[1][idx]),
                z=(0, mat[2][idx]),
                mode='lines',
                name=f'Vector {idx+1}' if names is None else names[idx],
                line=dict(color=colours[idx])
            )
            fig.add_cone(
                x=(mat[0][idx],),
                y=(mat[1][idx],),
                z=(mat[2][idx],),
                u=(0.2*mat[0][idx],),
                v=(0.2*mat[1][idx],),
                w=(0.2*mat[2][idx],),
                anchor='tip',
                hoverinfo='skip',
                colorscale=((0, colours[idx]), (1, colours[idx])),
                showscale=False
            )

    if showFig:
        fig.show()

##### Test Cases

In [ ]:
A = np.array([[2, -1, -2], [-4, 6, 3], [-4, -2, 8]])
B = np.array([[1, -4], [2, 3], [2, 2]])
C = np.array([[1, 1, 0], [0, 0, 3], [1, -1, 4]])
D = np.array([[8], [5], [3]])

W = mmread('data/football.mtx').toarray()     # 100x   https://www.cise.ufl.edu/research/sparse/matrices/Newman/football.html
X = mmread('data/delaunay_n10.mtx').toarray() # 1000x  https://www.cise.ufl.edu/research/sparse/matrices/DIMACS10/delaunay_n10.html
Y = mmread('data/msc01050.mtx').toarray()     # 1000x  https://www.cise.ufl.edu/research/sparse/matrices/Boeing/msc01050.html
Z = mmread('data/olm1000.mtx').toarray()      # 1000x  https://www.cise.ufl.edu/research/sparse/matrices/Bai/olm1000.html

<hr>

## Orthogonalization

### QR Factorization

Given an $m\times n$ Matrix $A$, there exists an $m\times m$ Orthogonal Matrix $Q$ and an $m\times n$ Upper Triangular Matrix $R$, such that $A=QR$.

$$A_{m\times n}=Q_{m\times m}R_{m\times n}$$

<hr>

### Householder's Method

<hr>

Given a Non-Zero Vector $x\neq e_1$, the Householder Matrix $H$ defined by the Vector $V$,

$$\begin{align*}H&=I-\dfrac{2VV^T}{V^TV}\\ V&=x\pm ||x||_2e_1\text{ such that}\\ Hx&=\mp ||x||_2e_1\end{align*}$$

Here, $e_1$ is the First Vector of an Identity Matrix of Order $n\times n$.

<hr>

In [ ]:
def getHouseholderMatrix(mat: np.ndarray):
    x = mat[:, 0].reshape(-1, 1)
    v = np.copy(x)
    v[0, 0] += np.sign(x[0, 0]) * np.linalg.norm(x)
    v = v / np.linalg.norm(v)
    h = np.identity(mat.shape[0]) - 2 * (v @ v.T)
    return h

def getQRfromHouseholder(mat: np.ndarray):
    hhs = []
    cols = mat.shape[0]
    for i in range(min(mat.shape)):
        hcap = getHouseholderMatrix(mat[i:, i:])
        hi = np.identity(cols)
        hi[i:, i:] = hcap
        mat = hi @ mat
        hhs.append(hi)

    q = np.identity(cols)
    for h in hhs[::-1]:
        q = h @ q

    return q, mat

#### Examples

In [ ]:
qHA, rHA = getQR(A, getQRfromHouseholder)

In [ ]:
qHB, rHB = getQR(B, getQRfromHouseholder)

In [ ]:
qHC, rHC = getQR(C, getQRfromHouseholder)

In [ ]:
qHW, rHW = getQR(W, getQRfromHouseholder, False)
np.all(np.isclose(qHW @ rHW, W))

In [ ]:
qHX, rHX = getQR(X, getQRfromHouseholder, False)
np.all(np.isclose(qHX @ rHX, X))

In [ ]:
qHY, rHY = getQR(Y, getQRfromHouseholder, False)
np.all(np.isclose(qHY @ rHY, Y))

In [ ]:
qHZ, rHZ = getQR(Z, getQRfromHouseholder, False)
np.all(np.isclose(qHZ @ rHZ, Z))

#### Properties

In [ ]:
h = getHouseholderMatrix(D)

prettyPrint(D, prefix='u=')
prettyPrint(h, prefix='H=')

$Hu=-u,H(-u)=u,H^2=I$

In [ ]:
hD = h @ D
hhD = h @ hD
hh = h @ h

prettyPrint(hD, prefix='Hu=')
prettyPrint(hhD, prefix='H(Hu)=', suffix='=u')
prettyPrint(hh, prefix='H^2=', suffix='=I')

In [ ]:
fig = go.Figure()
showMatrixAsVectors(D, fig, ('u',))
showMatrixAsVectors(hD, fig, ('Hu',), COLOURS[1:])
showMatrixAsVectors(hhD, fig, ('H(Hu)',), COLOURS[2:])
fig.show()

$H$ is Symmetric

In [ ]:
np.all(np.isclose(h, h.T))

$Hv=-v$

In [ ]:
v = D.copy()[:, 0].reshape(-1, 1)
v[0, 0] += np.sign(D[0, 0]) * np.linalg.norm(D)
v = v / np.linalg.norm(v)
hv = h @ v

prettyPrint(v, prefix='v=')
prettyPrint(hv, prefix='Hv=', suffix='=-v')

In [ ]:
fig = go.Figure()
showMatrixAsVectors(v, fig, ('v',))
showMatrixAsVectors(hv, fig, ('Hv',), COLOURS[1:])
fig.show()

#### Figures

In [ ]:
fig = go.Figure()
showMatrixAsVectors(A, fig, ('A1', 'A2', 'A3'))
showMatrixAsVectors(10*qHA, fig, ('Q1', 'Q2', 'Q3'), COLOURS[3:])
fig.show()

In [ ]:
fig = go.Figure()
showMatrixAsVectors(B, fig, ('B1', 'B2'))
showMatrixAsVectors(3*qHB, fig, ('Q1', 'Q2', 'Q3'), COLOURS[3:])
fig.show()

In [ ]:
fig = go.Figure()
showMatrixAsVectors(C, fig, ('C1', 'C2', 'C3'))
showMatrixAsVectors(3*qHC, fig, ('Q1', 'Q2', 'Q3'), COLOURS[3:])
fig.show()

### Givens Method

<hr>

A Matrix of the form

$$G(i,k,\theta)=\begin{cases}g_{xy}=+\cos\theta&\text{if }x=y=i\\ g_{xy}=+\sin\theta&\text{if }x=i,y=k\\ g_{xy}=+\cos\theta&\text{if }x=y=k\\ g_{xy}=-\sin\theta&\text{if }x=k,y=i\\ g_{xy}=1&\text{else if }x=y\\ g_{xy}=0&\text{otherwise}\end{cases}$$

for some $\theta$ is called a Givens Matrix.

<hr>

In [ ]:
def getGivensMatrix(n: int, m: int, i: int, k: int, theta: float = None, x: np.ndarray = None) -> np.ndarray:
    assert i != k, 'i and k must be different'
    assert n > 0 and m > 0, 'n and m must be greater than 0'
    assert i < n and i < m and k < n and k < m, 'i and k must be less than n and m'
    assert not (theta is None and x is None), 'Either theta or x must be provided'

    if theta is None:
        xi, xk = x[i], x[k]
        deno = np.sqrt(xi**2 + xk**2)
        cos = xi / deno
        sin = -xk / deno
    else:
        cos = np.cos(theta)
        sin = np.sin(theta)

    mat = np.eye(n, m)
    mat[i][i] = cos
    mat[k][k] = cos
    mat[i][k] = sin
    mat[k][i] = -sin

    return mat

def getQRfromGivens(a: np.ndarray) -> tuple[np.ndarray, np.ndarray]:
    n, m = a.shape    
    givens = []
    lowerIndices = ((i, j) for i in range(1, n) for j in range(i))
    for i, j in lowerIndices:
        if a[i][j] == 0:
            continue
        g = getGivensMatrix(n, m, j, i, x=a.T[j])
        a = g.T @ a
        givens.append(g)

    q = np.eye(n, m)
    for g in givens:
        q @= g

    return q, a

#### Examples

In [ ]:
qGA, rGA = getQR(A, getQRfromGivens)

In [ ]:
qGC, rGC = getQR(C, getQRfromGivens)

In [ ]:
qGW, rGW = getQR(W, getQRfromGivens, False)
np.all(np.isclose(qGW @ rGW, W))

In [ ]:
qGX, rGX = getQR(X, getQRfromGivens, False)
np.all(np.isclose(qGX @ rGX, X))

In [ ]:
qGY, rGY = getQR(Y, getQRfromGivens, False)
np.all(np.isclose(qGY @ rGY, Y))

In [ ]:
qGZ, rGZ = getQR(Z, getQRfromGivens, False)
np.all(np.isclose(qGZ @ rGZ, Z))

#### Figures

In [ ]:
fig = go.Figure()
showMatrixAsVectors(A, fig, ('A1', 'A2', 'A3'))
showMatrixAsVectors(10*qGA, fig, ('Q1', 'Q2', 'Q3'), COLOURS[3:])
fig.show()

In [ ]:
fig = go.Figure()
showMatrixAsVectors(C, fig, ('C1', 'C2', 'C3'))
showMatrixAsVectors(3*qGC, fig, ('Q1', 'Q2', 'Q3'), COLOURS[3:])
fig.show()

### Classical & Modified Gram Schmidt Method

<hr>

In [ ]:
import numpy as np

def getQRfromGramSchmidt(mat: np.ndarray) -> tuple[np.ndarray, np.ndarray]:
    m, n = mat.shape
    q, r = np.zeros((m, m)), np.zeros((n, n))

    for j in range(n):
        v = mat[:, j]
        for i in range(j):
            r[i, j] = q[:, i].T @ mat[:, j]
            v = v.squeeze() - (r[i, j] * q[:, i])
        r[j, j] = np.linalg.norm(v)
        q[:, j] = (v / r[j, j]).squeeze()

    return q, r

#### Examples

In [ ]:
qSA, rSA = getQR(A, getQRfromGramSchmidt)

In [ ]:
qSC, rSC = getQR(C, getQRfromGramSchmidt)

In [ ]:
qSW, rSW = getQR(W, getQRfromGramSchmidt, False)
np.all(np.isclose(qSW @ rSW, W))

In [ ]:
qSX, rSX = getQR(X, getQRfromGramSchmidt, False)
np.all(np.isclose(qSX @ rSX, X))

In [ ]:
qSY, rSY = getQR(Y, getQRfromGramSchmidt, False)
np.all(np.isclose(qSY @ rSY, Y))

In [ ]:
qSZ, rSZ = getQR(Z, getQRfromGramSchmidt, False)
np.all(np.isclose(qSZ @ rSZ, Z))

#### Figures

In [ ]:
fig = go.Figure()
showMatrixAsVectors(A, fig, ('A1', 'A2', 'A3'))
showMatrixAsVectors(10*qSA, fig, ('Q1', 'Q2', 'Q3'), COLOURS[3:])
fig.show()

In [ ]:
fig = go.Figure()
showMatrixAsVectors(C, fig, ('C1', 'C2', 'C3'))
showMatrixAsVectors(3*qSC, fig, ('Q1', 'Q2', 'Q3'), COLOURS[3:])
fig.show()

### Comparison Figures

<hr>

In [ ]:
fig = go.Figure()
showMatrixAsVectors(qHA, fig, ('H1', 'H2', 'H3'))
showMatrixAsVectors(qGA, fig, ('G1', 'G2', 'G3'), COLOURS[3:])
showMatrixAsVectors(qSA, fig, ('S1', 'S2', 'S3'), COLOURS[6:])
fig.show()

In [ ]:
fig = go.Figure()
showMatrixAsVectors(qHC, fig, ('H1', 'H2', 'H3'))
showMatrixAsVectors(qGC, fig, ('G1', 'G2', 'G3'), COLOURS[3:])
showMatrixAsVectors(qSC, fig, ('S1', 'S2', 'S3'), COLOURS[6:])
fig.show()

<hr>

## Least Squares Method

<hr>

In [ ]:
def leastSquareSolution(A: np.ndarray, b: np.ndarray) -> np.ndarray:
    m, n = A.shape

    if m >= n:
        inv = np.linalg.inv(A.T @ A)
        invAT = inv @ A.T
        x = invAT @ b
    else:
        inv = np.linalg.inv(A @ A.T)
        aTinv = A.T @ inv
        x = aTinv @ b

    return x

In [ ]:
xA = leastSquareSolution(A, D)
xB = leastSquareSolution(B, D)

In [ ]:
prettyPrint(A, prefix='A=')
prettyPrint(D, prefix='b=')
prettyPrint(xA, prefix='x=')

In [ ]:
prettyPrint(B, prefix='A=')
prettyPrint(D, prefix='b=')
prettyPrint(xB, prefix='x=')

<hr>